<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Step 1: Define Variational Autoencoder (VAE) Model
class VAE(nn.Module):
    def __init__(self, latent_dim=128):
        super(VAE, self).__init__()

        # Encoder: Convolutional layers to downsample the input
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),  # 32x32 -> 16x16
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), # 16x16 -> 8x8
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # 8x8 -> 4x4
            nn.ReLU(),
        )

        # Latent vectors `mu` and `log_var`
        self.fc_mu = nn.Linear(256*4*4, latent_dim)     # Mean of the latent space
        self.fc_log_var = nn.Linear(256*4*4, latent_dim) # Log-variance of the latent space

        # Decoder: Fully connected + transposed convolutions to upsample the latent representation
        self.fc_decoder = nn.Linear(latent_dim, 256*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 4x4 -> 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # 8x8 -> 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),     # 16x16 -> 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        """Encodes the input into latent space"""
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten the image to a vector
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        """Sample from a Gaussian distribution using the reparameterization trick"""
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        """Decodes the latent variable `z` back to an image"""
        x = self.fc_decoder(z)
        x = x.view(x.size(0), 256, 4, 4)  # Reshape to image size
        x = self.decoder(x)
        return x

    def forward(self, x):
        """Forward pass through VAE"""
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        reconstructed = self.decode(z)
        return reconstructed, mu, log_var

# Step 2: Define the loss function (Reconstruction loss + KL Divergence)
# def vae_loss(reconstructed, original, mu, log_var):
#     # Reconstruction loss (binary cross entropy)
#     recon_loss = nn.functional.binary_cross_entropy(reconstructed, original, reduction='sum')

#     # KL Divergence loss (encourages the latent space to be Gaussian)
#     kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

#     return recon_loss + kl_divergence


def vae_loss(reconstructed, original, mu, log_var):
    # Reconstruction loss using Mean Squared Error (MSE)
    recon_loss = nn.functional.mse_loss(reconstructed, original, reduction='sum')

    # KL Divergence loss (encourages the latent space to be Gaussian)
    kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    return recon_loss + kl_divergence


In [3]:
# Step 3: Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Step 4: Initialize VAE model, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(latent_dim=128).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

100%|██████████| 170498071/170498071 [00:04<00:00, 36181240.81it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Step 5: Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, _ = data
        inputs = inputs.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass through the model
        reconstructed, mu, log_var = model(inputs)

        # Compute VAE loss
        loss = vae_loss(reconstructed, inputs, mu, log_var)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 2:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], Loss: {loss.item()}')

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(trainloader)}')

print('Finished Training')

# Step 6: Sample from the latent space to generate new images
def sample_from_latent_space(model, num_samples=5):
    model.eval()
    with torch.no_grad():
        z = torch.randn(num_samples, 128).to(device)  # Sample random latent vectors
        generated_images = model.decode(z)
        generated_images = generated_images.cpu()

    # Plot the generated images
    plt.figure(figsize=(10, 2))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(generated_images[i].permute(1, 2, 0).numpy())
        plt.axis('off')
    plt.show()

# Generate new images
sample_from_latent_space(model, num_samples=5)

Epoch [1/10], Step [3/782], Loss: 43622.16796875
Epoch [1/10], Step [103/782], Loss: 38468.92578125
Epoch [1/10], Step [203/782], Loss: 42971.65625
Epoch [1/10], Step [303/782], Loss: 39210.34765625
Epoch [1/10], Step [403/782], Loss: 38029.515625
Epoch [1/10], Step [503/782], Loss: 39319.15625
Epoch [1/10], Step [603/782], Loss: 42106.17578125
Epoch [1/10], Step [703/782], Loss: 31534.890625
Epoch [1/10], Loss: 37864.44105908328
Epoch [2/10], Step [3/782], Loss: 35585.0859375
Epoch [2/10], Step [103/782], Loss: 33536.5
Epoch [2/10], Step [203/782], Loss: 38649.703125
Epoch [2/10], Step [303/782], Loss: 35493.49609375
Epoch [2/10], Step [403/782], Loss: 34420.13671875
Epoch [2/10], Step [503/782], Loss: 33296.421875
Epoch [2/10], Step [603/782], Loss: 33221.72265625
Epoch [2/10], Step [703/782], Loss: 39455.96875
Epoch [2/10], Loss: 35133.885086566894
Epoch [3/10], Step [3/782], Loss: 32602.884765625
Epoch [3/10], Step [103/782], Loss: 35598.67578125
Epoch [3/10], Step [203/782], Loss: